<a href="https://colab.research.google.com/github/dariocastillo11/ComunicacionesDigitales/blob/main/laboratorio2SDR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#---------------------------------------------------------------------------------
#Project     : lABORTORIO N° 2 Caracterización del ruido del
#                               canal inhalámbrico.
# Authors    : Castillo, Darío Alberto
# E-mail     : dario.castillo@mi.unc.edu.ar
#---------------------------------------------------------------------------------

In [5]:
!pip install pyadi-iio
import adi
!pip install matplotlib
!pip install scikit-dsp-comm
!pip install scikit-commpy
#------------------------------- Libraries -------------------------------
import numpy as np
import matplotlib.pyplot as plt

#punto 5
from scipy.stats import norm

#punto 6
!pip install statsmodels
import statsmodels.api as sm
import scipy.stats as stats

AttributeError: /usr/bin/python3: undefined symbol: iio_get_backends_count

In [ ]:
#------------------------------- SDR Parameter Configuration -------------------------------

Uri = "ip:192.168.1.33"
sdr = adi.Pluto(Uri)

Loopback               = 0 # El loopback permite que la señal transmitida
#se reciba nuevamente, lo cual no es deseado en este caso.
sdr.loopback = Loopback


#configuracion de ganancia del transmisor
TxAtten = -70 #valor maximo de atenuacion
sdr.tx_hardwaregain_chan0  = TxAtten # atenuacion, valid range is -90 to 0 dB [dB]

#configuracion de ganancia del receptor
RxHardwareGain     = 70
sdr.rx_hardwaregain_chan0   = RxHardwareGain

#configuracion de la frecuencia del transmisor
TxLOFreq         = 918e6
sdr.tx_lo = int(TxLOFreq)

#configuracion de la frecuencia del receptor
RxLOFreq         = TxLOFreq      # Carrier frequency of RX path [Hz]
sdr.rx_lo = int(RxLOFreq) # Carrier frequency of RX path [Hz]

#configuracion de ancho de banda de transmision
TxRfBw           = 2.0e6
sdr.tx_rf_bandwidth = int(TxRfBw)

#configuracion de ancho de banda de recepcion
RxRfBw           = 2.0e6
sdr.rx_rf_bandwidth = int(RxRfBw)

#configuracion de la frecuencia de muestreo
samplingRate = 2.0e6  # Frecuencia de muestreo [Samples/Sec] samples=muestras
sdr.sample_rate = int(samplingRate)

#lectura de datos
nSamples = 2**20  # Número de muestras a adquirir/recibir
RxBufferSize = nSamples
sdr.rx_buffer_size = RxBufferSize
received_signal = sdr.rx()



Tx_cyclic_buffer      = False
GainControlModes = "manual"






In [ ]:
# Configurar la señal de transmisión a cero (sin transmisión)



señalrecibida = sdr.rx()  # Recibe la señal del SDR

In [ ]:
#punto 1

#calculo parte real e imaginaria de la señal
parteRealSeñal=np.real(señalrecibida)
parteImaginariaSeñal=np.imag(señalrecibida)

#calculo la frecuencia de la señal
frecuenciaR = np.fft.fftfreq(len(parteRealSeñal), d=1/samplingRate)
frecuenciaI = np.fft.fftfreq(len(parteImaginariaSeñal), d=1/samplingRate)

# Calculo la transformada rápida de Fourier  para la componente real e imaginaria
transformadaFourierReal = np.fft.fft(parteRealSeñal)
transformadaFourierImaginaria = np.fft.fft(parteImaginariaSeñal)

# Función para graficar la señal en tiempo y frecuencia
def plotSignal(señalrecibida, samplingRate):
    # Grafica en el dominio del tiempo
    plt.figure(figsize=(20,8), dpi=80, facecolor='w', edgecolor='k')


    # Componente en fase (Real)
    plt.subplot(2,1,1)
    plt.plot(parteRealSeñal)
    plt.xlabel('Muestra')
    plt.ylabel('Amplitud')
    plt.title('Componente en fase (Real) del ruido')
    plt.grid()

    # Componente en cuadratura (Imaginaria)
    plt.subplot(2,1,2)
    plt.plot(parteImaginariaSeñal)
    plt.xlabel('Muestra')
    plt.ylabel('Amplitud')
    plt.title('Componente en cuadratura (Imaginaria) del ruido')
    plt.grid()

    plt.tight_layout()
    plt.show()
#########################################################################################################
    # Grafica en el dominio de la frecuencia:
    plt.figure(figsize=(20,8), dpi=80, facecolor='w', edgecolor='k')
    # Graficar la magnitud de la FFT de la componente real (espectro de frecuencia)
    plt.subplot(2,1,1)
    plt.plot(frecuenciaR[:len(frecuenciaR)//2], np.abs(transformadaFourierReal)[:len(transformadaFourierReal)//2])
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('Magnitud')
    plt.title('Espectro de Frecuencia de la Componente Real')
    plt.grid()
    # Grafica de la magnitud de la FFT de la componente imaginaria (espectro de frecuencia)
    plt.subplot(2,1,2)
    plt.plot(frecuenciaI[:len(frecuenciaI)//2], np.abs(transformadaFourierImaginaria)[:len(transformadaFourierImaginaria)//2])
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('Magnitud')
    plt.title('Espectro de Frecuencia de la Componente Imaginaria')
    plt.grid()
    plt.tight_layout()
    plt.show()

# Recibe la señal del SDR cargada en 'rxSignal'
# Grafica de las componentes real e imaginaria del ruido y el espectro de frecuencia por separado
plotSignal(señalrecibida, samplingRate)

In [ ]:
#punto 2
#  Obtener la media y la varianza.
N = len(señalrecibida)  # Número de muestras
##############################################################
# Calcular la media de la parte real de la señal
sum_real = 0
for i in range(N):
    sum_real += parteRealSeñal[i]
mediaParteReal = sum_real / N  # Media de la parte real
##############################################################
# Calcular la varianza de la parte real de la señal
sumaDiferenciaCuadradoReal = 0
for i in range(N):
    sumaDiferenciaCuadradoReal += (parteRealSeñal[i] - mediaParteReal) ** 2
varianzaParteReal = sum_squared_diff_real / N  # Varianza de la parte real
##############################################################
# Calcular la media de la parte imaginaria de la señal
sum_imag = 0
for i in range(N):
    sum_imag += parteImaginariaSeñal[i]
mediaParteImaginaria = sum_imag / N  # Media de la parte imaginaria
##############################################################
# Calcular la varianza de la parte imaginaria de la señal
sumaDiferenciaCuadradoImaginaria = 0
for i in range(N):
    sumaDiferenciaCuadradoImaginaria += (parteImaginariaSeñal[i] - mediaParteImaginaria) ** 2
varianzaParteImaginaria = sumaDiferenciaCuadradoImaginaria / N  # Varianza de la parte imaginaria
##############################################################
# Imprimir los resultados
print(f"Media Parte Real: {mediaParteReal}")
print(f"Varianza Parte Real: {varianzaParteReal}")
print(f"Media Parte Imaginaria: {mediaParteImaginaria}")
print(f"Varianza Parte Imaginaria: {varianzaParteImaginaria}")


In [ ]:
# Graficar el histograma

#punto 3

def plotHistogram(señalrecibida, bins=500):
    plt.figure(figsize=(10,6))
    plt.subplot(2,1,1)
    plt.hist(parteRealSeñal, bins=bins, density=True, alpha=0.5, label='Componente Real')
    plt.xlabel('Amplitud')
    plt.ylabel('Frecuencia')
    plt.title('Histograma de la señal parte real')
    plt.legend(loc='best')
    plt.grid()
    plt.show()

    plt.figure(figsize=(10,6))
    plt.subplot(2,1,1)
    plt.hist(parteImaginariaSeñal, bins=bins, density=True, alpha=0.5, label='Componente Imaginaria')
    plt.xlabel('Amplitud')
    plt.ylabel('Frecuencia')
    plt.title('Histograma de la señal parte imaginaria')
    plt.legend(loc='best')
    plt.grid()
    plt.show()

#grafica  del histograma
plotHistogram(señalrecibida)


In [ ]:
#punto 5
# Superponer el histograma normalizado con una función de densidad de probabilidad (PDF) gaussiana teórica.

def plotHistogramWithPDF(signal):
    # Estimación de los parámetros de la gaussiana para la componente real
    mean_real = np.mean(np.real(signal))
    std_real = np.std(np.real(signal))

    # Estimación de los parámetros de la gaussiana para la componente imaginaria
    mean_imag = np.mean(np.imag(signal))
    std_imag = np.std(np.imag(signal))

    # Grafica del  histograma para la componente real e imaginaria
    plt.figure(figsize=(10,6))
    plt.hist(np.real(signal), bins=100, density=True, alpha=0.5, label='Componente Real')
    plt.hist(np.imag(signal), bins=100, density=True, alpha=0.5, label='Componente Imaginaria')

    # Función de densidad de probabilidad  gaussiana teórica para la componente real
    x_real = np.linspace(np.min(np.real(signal)), np.max(np.real(signal)), 1000)
    pdf_real = norm.pdf(x_real, mean_real, std_real)
    plt.plot(x_real, pdf_real, label='PDF Gaussiana Teórica (Real)', color='black', linewidth=2)

    # Función de densidad de probabilidad gaussiana teórica para la componente imaginaria
    x_imag = np.linspace(np.min(np.imag(signal)), np.max(np.imag(signal)), 1000)
    pdf_imag = norm.pdf(x_imag, mean_imag, std_imag)
    plt.plot(x_imag, pdf_imag, label='PDF Gaussiana Teórica (Imaginaria)', color='red', linewidth=2)

    plt.xlabel('Amplitud')
    plt.ylabel('Frecuencia')
    plt.title('Histograma con PDF Gaussiana Teórica')
    plt.legend(loc='best')
    plt.grid()
    plt.show()

# Llamar a la función para graficar
plotHistogramWithPDF(rxSignal)


In [ ]:
#punto 6
#Analizar la gaussianidad mediante un gráfico Q-Q haciendo uso de la función propia de python "qqplot".

def plotQQPlot(signal):
    sm.qqplot(np.real(signal), line ='45')
    plt.title('Q-Q Plot Componente Real')
    plt.grid()
    plt.show()

    sm.qqplot(np.imag(signal), line ='45')
    plt.title('Q-Q Plot Componente Imaginaria')
    plt.grid()
    plt.show()

# Llamar a la función para el gráfico Q-Q
plotQQPlot(rxSignal)

In [ ]:
#punto 7
# Generar mediante un código, una variable gaussiana compleja que tenga la misma media
# y varianza que la señal muestreada y obtener su gráfica temporal,
# su gráfica en frecuencia y su histograma.

# Función para graficar la señal en el tiempo y en la frecuencia
def plotSignalTimeFreq(signal, samplingRate):
    # Grafica de la señal en el tiempo
    plt.figure(figsize=(10, 6))
    plt.subplot(2, 1, 1)
    plt.plot(np.real(signal), label="Componente Real")
    plt.plot(np.imag(signal), label="Componente Imaginaria")
    plt.xlabel('Muestra')
    plt.ylabel('Amplitud')
    plt.title('Señal Gaussiana Compleja en el Tiempo')
    plt.legend()
    plt.grid()

    # Grafica de la señal en el dominio de frecuencia
    plt.subplot(2, 1, 2)
    # Transformada de Fourier
    signal_freq = np.fft.fft(signal)
    freq = np.fft.fftfreq(len(signal), d=1/samplingRate)
    # Grafica de la magnitud en frecuencia
    plt.plot(freq[:len(freq)//2], np.abs(signal_freq)[:len(signal_freq)//2])
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('Magnitud')
    plt.title('Espectro de Frecuencia de la Señal Gaussiana Compleja')
    plt.grid()

    plt.tight_layout()
    plt.show()
# Calcular la media y varianza para la parte real e imaginaria
#modificar para calcularlo con formula y no codigo
mean_real = np.mean(np.real(rxSignal))
std_real = np.std(np.real(rxSignal))
mean_imag = np.mean(np.imag(rxSignal))
std_imag = np.std(np.imag(rxSignal))

# Generar señal gaussiana compleja con la misma media y varianza que la señal muestreada
#utilice funcion. preguntar si debo calcular de formula
complex_gaussian_signal = np.random.normal(mean_real, std_real, nSamples) + 1j * np.random.normal(mean_imag, std_imag, nSamples)

# Grafica de la  señal gaussiana compleja en el tiempo y frecuencia
plotSignalTimeFreq(complex_gaussian_signal, samplingRate)

# Grafica del  histograma de la señal compleja
plotHistogram(complex_gaussian_signal)
